In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import json
import cv2
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fold = r'/content/drive/MyDrive/Модель МО с использованием КЗ ИКБ-34/Обучение Модели МО/' #Может быть любой другой путь, главное, чтобы там были data{i}.json

In [ ]:
# data1 = {
#          'data': [],
#      'count_of_fingers': []
#  }

#  for i in range(19):
#      with open(f"{fold}data{i}.json", 'r', encoding='utf-8') as f:
#          data_json = json.load(f)
#          data1['data'].extend(data_json['data'])
#          data1['count_of_fingers'].extend(data_json['count_of_fingers'])

In [ ]:
#  data2 = {
#      'data': [],
#      'count_of_fingers': []
#  }

#  for i in range(19, 25):
#      with open(f"{fold}data{i}.json", 'r', encoding='utf-8') as f:
#          data_json = json.load(f)
#          data2['data'].extend(data_json['data'])
#          data2['count_of_fingers'].extend(data_json['count_of_fingers'])

In [ ]:
#  data3 = {
#      'data': [],
#      'count_of_fingers': []
#  }

#  for i in range(25, 33):
#      with open(f"{fold}data{i}.json", 'r', encoding='utf-8') as f:
#          data_json = json.load(f)
#          data3['data'].extend(data_json['data'])
#          data3['count_of_fingers'].extend(data_json['count_of_fingers'])
data = {
    'data': [],
    'count_of_fingers': []
}

for i in range(33):
    with open(f"{fold}data{i}.json", 'r', encoding='utf-8') as f:
        data_json = json.load(f)
        data['data'].extend(data_json['data'])
        data['count_of_fingers'].extend(data_json['count_of_fingers'])

In [ ]:
# print(data1['data'][0])
# print(data1['count_of_fingers'][0])
print(len(data['data']))
# print(len(data1['data']))

10468


In [ ]:
# print(len(data2['data']))

In [ ]:
# print(len(data3['data']))

In [ ]:
# data = {
#      'data_new' : [],
#      'data': [],
#      'cnt': []
# }

In [ ]:
# data['data'] = data1['data'] + data2['data'] + data3['data']
# data['cnt'] = data1['count_of_fingers'] + data2['count_of_fingers'] + data3['count_of_fingers']

In [ ]:
# data['data'] = np.abs(data['data'])
data['data'] = np.array(data['data'])
data['count_of_fingers'] = np.array(data['count_of_fingers'])
data['data'] = np.abs(data['data'])
data['data_new'] = []

In [ ]:
# for i in range(data['data'].shape[0]):
#     matr = data['data'][i]
#     gray = (255 - (matr * 255)).astype(np.uint8)
#     (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#     gray = cv2.resize(gray, (300, 300))
#     img = gray / 255.0

#     img = np.array(img).reshape(-1, 300 * 300, 1)
#     data['data_new'].append(img)
# Предобработка данных
for i in range(data['data'].shape[0]):
    matr = data['data'][i]
    gray = (255 - (matr * 255)).astype(np.uint8)
    (_, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    gray = cv2.resize(gray, (150, 150))
    img = gray / 255.0
    img = np.array(img).reshape(150, 150, 1)
    data['data_new'].append(img)

In [ ]:
data['data_new'] = np.array(data['data_new'])

In [ ]:
data['data_new'].shape

(10468, 150, 150, 1)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(data['data_new'], data['cnt'], test_size=0.2, random_state=42, stratify=data['cnt'])
# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(data['data_new'], data['count_of_fingers'], test_size=0.2, random_state=42, stratify=data['count_of_fingers'])

In [ ]:
y_train_oh = to_categorical(y_train, num_classes=6)
y_val_oh = to_categorical(y_val, num_classes=6)

In [ ]:
# prompt: выведи сколько у обучающих данных в сумме 0 пальцев, 1 палец, 2 пальца и т.п. (ключ [count_of_fingers])

from collections import Counter

finger_counts = Counter(data['count_of_fingers'])

for count, frequency in finger_counts.items():
    print(f"Number of images with {count} fingers: {frequency}")

Number of images with 1 fingers: 2036
Number of images with 2 fingers: 2196
Number of images with 3 fingers: 1748
Number of images with 4 fingers: 1608
Number of images with 5 fingers: 1592
Number of images with 0 fingers: 1288


In [ ]:
# input_shape = X_train.shape

In [ ]:
# input_shape

In [ ]:
# # inp = layers.Input(shape=(input_shape[1], input_shape[2], input_shape[3], ))
# inp = layers.Input(shape=input_shape)

# conv1 = layers.Conv2D(16, (3, 3), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.002))(inp)
# conv1 = layers.BatchNormalization()(conv1)  # Нормализация
# conv1 = layers.ReLU()(conv1)  # ReLU после нормализации
# conv1 = layers.Conv2D(16, (2, 2), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.002))(conv1)
# conv1 = layers.BatchNormalization()(conv1)  # Нормализация
# conv1 = layers.ReLU()(conv1)  # ReLU после нормализации
# pool1 = layers.MaxPooling2D((2, 2), padding ='same')(conv1)
# drout1 = layers.Dropout(0.3)(pool1)

# conv2 = layers.Conv2D(32, (2, 2), activation=None, padding='same')(drout1)
# conv2 = layers.BatchNormalization()(conv2)
# conv2 = layers.ReLU()(conv2)
# conv2 = layers.Conv2D(32, (2, 2), activation=None, padding='same')(conv2)
# conv2 = layers.BatchNormalization()(conv2)
# conv2 = layers.ReLU()(conv2)
# pool2 = layers.MaxPooling2D((2, 2), padding ='same')(conv2)
# drout2 = layers.Dropout(0.5)(pool2)

# transp3 = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(drout2)
# upsample = layers.UpSampling2D(size=(2, 2))(drout2)
# conv10 = layers.Conv2D(16, (3, 3), padding='same')(upsample)
# concat1 = layers.concatenate([transp3, conv10])
# conv8 = layers.Conv2D(32, (2, 2), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.003))(concat1)
# conv8 = layers.BatchNormalization()(conv8)
# conv8 = layers.ReLU()(conv8)
# conv8 = layers.Conv2D(16, (2, 2), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.002))(conv8)
# conv8 = layers.BatchNormalization()(conv8)
# conv8 = layers.ReLU()(conv8)
# drout3 = layers.Dropout(0.4)(conv8)

# flat = layers.Flatten()(drout3)

# dense1 = layers.Dense(16, activation=None, kernel_regularizer=regularizers.l2(0.001))(flat)
# dense1 = layers.BatchNormalization()(dense1)
# dense1 = layers.ReLU()(dense1)

# dense2 = layers.Dense(32, activation=None)(dense1)
# dense2 = layers.BatchNormalization()(dense2)
# dense2 = layers.ReLU()(dense2)

# dropout3 = layers.Dropout(0.5)(dense2)

# dense3 = layers.Dense(64, activation=None)(dropout3)
# dense3 = layers.BatchNormalization()(dense3)
# dense3 = layers.ReLU()(dense3)

# dropout1 = layers.Dropout(0.5)(dense3)

# dense4 = layers.Dense(128, activation=None, kernel_regularizer=regularizers.l2(0.002))(dropout1)
# dense4 = layers.BatchNormalization()(dense4)
# dense4 = layers.ReLU()(dense4)

# dropout2 = layers.Dropout(0.4)(dense4)

# dense7 = layers.Dense(64, activation=None, kernel_regularizer=regularizers.l2(0.002))(dropout2)
# dense7 = layers.BatchNormalization()(dense7)
# dense7 = layers.ReLU()(dense7)

# dense8 = layers.Dense(32, activation=None)(dense7)
# dense8 = layers.BatchNormalization()(dense8)
# dense8 = layers.ReLU()(dense8)

# dense9 = layers.Dense(16, activation=None)(dense8)
# dense9 = layers.BatchNormalization()(dense9)
# dense9 = layers.ReLU()(dense9)

# out = layers.Dense(6, activation='softmax')(dense9)
# Архитектура модели
inp = layers.Input(shape=(150, 150, 1))

conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0005))(inp)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0005))(conv1)
conv1 = layers.BatchNormalization()(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0005))(pool1)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0005))(conv2)
conv2 = layers.BatchNormalization()(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2)

conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0005))(pool2)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0005))(conv3)
conv3 = layers.BatchNormalization()(conv3)
pool3 = layers.MaxPooling2D((2, 2))(conv3)

attention = layers.GlobalAveragePooling2D()(pool3)
attention = layers.Dense(128, activation='relu')(attention)
attention = layers.Dense(128, activation='sigmoid')(attention)
attention_output = layers.Multiply()([pool3, attention])

flat = layers.Flatten()(attention_output)

dense1 = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(flat)
dense1 = layers.Dropout(0.5)(dense1)

dense2 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(dense1)
dense2 = layers.Dropout(0.5)(dense2)

out = layers.Dense(6, activation='softmax')(dense2)


In [ ]:
# bknd.clear_session()
model = models.Model(inputs=inp, outputs=out)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 150, 150, 32)         320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 150, 150, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv2d_1 (Conv2D)           (None, 150, 150, 32)         9248      ['batch_normalization[0][0

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr = 1e-6)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Коллбэки
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train_oh.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val_oh.shape)


X_train shape: (8374, 150, 150, 1)
y_train shape: (8374, 6)
X_val shape: (2094, 150, 150, 1)
y_val shape: (2094, 6)


In [ ]:
history = model.fit(
    X_train,
    y_train_oh,
    epochs = 150,
    batch_size = 32,
    validation_data = (X_val, y_val_oh),
    callbacks = [reduce_lr, early_stopping]
)

Epoch 1/150
262/262 [==============================] - 207s 780ms/step - loss: 2.6655 - accuracy: 0.2028 - val_loss: 2.2987 - val_accuracy: 0.2058 - lr: 0.0010
Epoch 2/150
262/262 [==============================] - 199s 762ms/step - loss: 2.2088 - accuracy: 0.2160 - val_loss: 2.1174 - val_accuracy: 0.2087 - lr: 0.0010
Epoch 3/150
262/262 [==============================] - 200s 765ms/step - loss: 2.0628 - accuracy: 0.2284 - val_loss: 2.0202 - val_accuracy: 0.2311 - lr: 0.0010
Epoch 4/150
262/262 [==============================] - 199s 761ms/step - loss: 1.9452 - accuracy: 0.2367 - val_loss: 1.8889 - val_accuracy: 0.2459 - lr: 0.0010
Epoch 5/150
262/262 [==============================] - 199s 760ms/step - loss: 1.8596 - accuracy: 0.2818 - val_loss: 1.8391 - val_accuracy: 0.2698 - lr: 0.0010
Epoch 6/150
262/262 [==============================] - 198s 754ms/step - loss: 1.8113 - accuracy: 0.3229 - val_loss: 2.8245 - val_accuracy: 0.2779 - lr: 0.0010
Epoch 7/150
262/262 [===================

In [ ]:
y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1)
accuracy_score(y_pred, y_val)

66/66 [==============================] - 8s 123ms/step


0.625119388729704

In [ ]:
y_pred = model.predict(X_train)
y_pred = np.argmax(y_pred, axis=1)
accuracy_score(y_pred, y_train)

262/262 [==============================] - 32s 121ms/step


0.8359207069500836

In [ ]:
model.save(fold + 'modelzzz.h5')

In [ ]:
print("TensorFlow version:", tf.__version__)
import sys
print("Python version:", sys.version)

TensorFlow version: 2.15.0
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
